# Desarrollo de STFT
***

## Integrantes

-Felipe Véliz ; felipe.veliz@alumnos.uach.cl

-Leonardo Santos ; leonardo.santos@alumnos.uach.cl

## Institución

Universidad Austral de Chile

## Profesor

- Victor Poblete

## Asignatura

ACUS099

## Objetivo

El objetivo de este trabajo es desarrollar la transformada de fourier de tiempo corto de forma discreta, usando lenguaje de programación python.

# Librerias a usar

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, ifft, fftshift
import IPython.display as disp 
from scipy.signal import chirp
from scipy.signal.windows import hann,hamming

## Preparando la señal

Para demsotrar como funciona la STFT, necesitamos una señal inicial con la cual trabajar.


In [ ]:
sr=44100 #Sample rate 
t_fin=10  #Duracion de la señal en segundos
t=t_fin*sr  #Duracion de la señal en muestras
T=np.linspace(0,t_fin,t) #vector de tiempo
sig= chirp(T, f0=50, f1=7000, t1=t_fin, method='linear')  #señal usando chirp
disp.Audio(sig, rate=sr)

In [ ]:
len_f=1024  #largo de la ventana 
overlap= 0.5  #porcentaje de traslape

num_f=int(round(len(sig)/(len_f*(1-overlap)))) #numero de ventanas en funcion de la señal
count_f= len_f*num_f    #cantidad total de la muestras de la matriz

print("La señal posee un total de",num_f, "ventanas")

## Aplicación de función ventana

Al usar cierto porcentaje de overlap o traslape nos encontramos con un problema, tenemos datos duplicados en las ventanas, lo cual provoca un incremento de amplitud. 

Las funciones ventana son ponderaciones usadas para solucionar este problema, cada una nivela de manera distinta el incremento de amplitud.  

In [ ]:
tp=0.5
y=np.zeros(len(sig[0:int(tp*sr)]))
plt.figure(figsize=(15,5))
plt.plot(T[0:int(tp*sr)],sig[0:int(tp*sr)])
plt.plot(T[0:int(tp*sr)],y)
plt.xlabel("Tiempo [s]",size=14)
plt.ylabel("Amplitud",size=14)
plt.title("Forma de onda temporal",size=15)
plt.show()

In [ ]:
#primera ventana
plt.figure(figsize=(15,5))
plt.plot(T[0:len_f],sig[0:len_f])

### Funciones ventana

A continuacion, usaremos dos ponderaciones distintas y las aplicaremos en la primera ventana que contiene los datos de la señal, para ver como estas afectan de manera distinta.

### Funciones definidas  matemáticamente:

#### Función Hamming

 $$
\begin{align}
w(n) = 0.54 - 0.46cos\left(\frac{2\pi n}{M-1}\right), \qquad 0\le n \le M-1
\end{align}
$$
***

#### Función Hann

 $$
\begin{align}
w(n) = 0.5 - 0.5cos\left(\frac{2\pi n}{M-1}\right), \qquad 0\le n \le M-1
\end{align}
$$

In [ ]:
ham=hamming(len_f)
han=hann(len_f)


plt.figure(figsize=(14,5))
plt.plot(ham,label="hamming")
plt.plot(han,label="hann")
plt.title("Comparación de ponderaciones",size=15)
plt.legend()
plt.show()

In [ ]:

mat_f=[]                   #la señal se va separando en ventanas de manera reiterativa dentro del ciclo for
for i in range(num_f):     #hasta el total de ventanas -1
    f=list(sig[int(len_f*(1-overlap))*i:int(len_f*((1-overlap)*i+1))])
    while len(f)<len_f:   #esta condicion identifica cuando nos faltan datos en la ventana y en ese caso rellena con 0 
        f.append(0)    
    mat_f.append(f)

mat_f_np=np.array(mat_f).T
mm_ham=(mat_f_np.T*ham).T
mm_han=(mat_f_np.T*han).T
mm_ham.shape

In [ ]:
f=50 #numero de ventana
tf1=int(len_f*(1-overlap))*f
tf2=int(len_f*((1-overlap)*f+1)) 
f_50=tf2-len_f #Posición de la ventana 50 en muestras
sig_f=sig[f_50:f_50+len_f]

plt.figure(figsize=(15,5))
plt.plot(mm_ham[:,f])
plt.plot(mm_han[:,f])
plt.plot(sig_f)

print(tf1,tf2,"muestras") #Ubicación de la ventana en muestras
print(tf1/sr,(tf2/sr),'segundos') #Ubicación de la ventana en segundos
print(f_50)
plt.show()

Como podemos aprecias, la función Hann afecta mas a los extremos de la ventana.

## Aplicar la STFT

Para aplicar STFT crearemos un arreglo de matriz con numpy y luego le aplicaremos la funcion ventana *Hamming*.
Finalmente aplicaremos la funcion *fftshift* para tener solamente la parte positiva de las frecuencias.

In [ ]:
fft_ii=[]
for i in range(mm_ham.shape[1]):
    f_i = mm_ham[:,i]
    ff_i= ((fftshift(fft(f_i,n=len_f))[len_f//2:]))
    fft_ii.append(np.abs(ff_i))

print(len(fft_ii))
m_fft=np.asarray(fft_ii).T
print(m_fft.shape)

fr=np.linspace(0,sr//2,num=m_fft.shape[1])
tiempo=np.linspace(0,t_fin,num=m_fft.shape[0])

plt.figure(figsize=(15,10))
plt.pcolormesh(m_fft)  
plt.xlabel('tiempo [ventanas]',size=14)
plt.ylabel("frecuencia [bin]",size=14)

## Conclusion 

En este trabajo
